In [20]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen

# Load county shapefiles
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [21]:
# Load datasets

# Zillow house prices
zhvi = pd.read_csv("/home/kbarone/capp30122/proj-bust_a_move/zillow_home_value_index_county.csv", index_col="RegionID")

# Median income and poverty rate
med_inc_pov = pd.read_csv("/home/kbarone/capp30122/proj-bust_a_move/census_2020_median_inc_and_poverty.csv", dtype={'state': object, 'fips_county': object})

# Zillow RegionID to County FIPS code crosswalk
crosswalk = pd.read_csv("/home/kbarone/capp30122/proj-bust_a_move/CountyCrossWalk_Zillow.csv", encoding = "ISO-8859-1", index_col="CountyRegionID_Zillow", dtype={"FIPS" : object})

# Population estimates
population = pd.read_csv("/home/kbarone/capp30122/proj-bust_a_move/county_population.csv", encoding = "ISO-8859-1", dtype={"STATE" : object, "COUNTY" : object})

# Mobility data
mobility = pd.read_csv("/home/kbarone/capp30122/proj-bust_a_move/google_mobility_county.csv", dtype= {"countyfips": str})

# Race data
#race = pd.read_csv("/Users/katybarone/Documents/uchicago/race_by_county.csv")

In [22]:
med_inc_pov["state"] = med_inc_pov['state'].str.zfill(2)
med_inc_pov['County Code'] = med_inc_pov["state"] + med_inc_pov["fips_county"]
med_inc_pov["county"] = med_inc_pov['state'].str.zfill(3)

In [23]:
population["STATE"] = population["STATE"].str.zfill(2)
population["COUNTY"] = population["COUNTY"].str.zfill(3)
population["COUNTY_FIPS"] = population["STATE"] + population["COUNTY"]

In [24]:
years = zhvi.loc[:, "2019-01-31":"2021-12-31":1]

regions = zhvi.loc[:, "SizeRank":"MunicipalCodeFIPS":1]

zhvi2 = regions.join(years)

zhvi2["2019_average"] = zhvi2.loc[:, "2019-01-31":"2019-12-31": 1].mean(axis=1)
zhvi2["2020_average"] = zhvi2.loc[:, "2020-01-31":"2020-12-31": 1].mean(axis=1)
zhvi2["2021_average"] = zhvi2.loc[:, "2021-01-31":"2021-12-31": 1].mean(axis=1)
zhvi2["2020_increase"] = ((zhvi2["2020_average"] - zhvi2["2019_average"])/zhvi2["2019_average"])*100
zhvi2["2021_increase"] = ((zhvi2["2021_average"] - zhvi2["2020_average"])/zhvi2["2020_average"])*100
zhvi2["2021_2yr_increase"] = ((zhvi2["2021_average"] - zhvi2["2019_average"])/zhvi2["2019_average"])*100

zhvi2["text_20"] = " "
zhvi2["text_21"] = " "
zhvi2["text_2yrs"] = " "

zhvi2.dropna()

zhvi_county = pd.merge(zhvi2, crosswalk, left_index=True, right_index=True)
zhvi_county["FIPS"] = zhvi_county["FIPS"].str.zfill(5)
zhvi_county_inc = pd.merge(zhvi_county, med_inc_pov, left_on="FIPS", right_on="County Code")
zhvi_county_inc_pop = pd.merge(zhvi_county_inc, population, left_on="FIPS", right_on="COUNTY_FIPS")

# Format text for hovering
for idx, row in zhvi_county_inc_pop.iterrows():
    zhvi_county_inc_pop.at[idx, 'text_20'] = 'County:' + row["RegionName"] + '<br>' + 'State:' + row["State"] + \
    '<br>' + '2019-20 increase:' + str(round(row["2020_increase"], 3)) + '<br>' + 'Med_Inc:' + str(row["med_inc"]) + \
        '<br>' + 'Pop_2019:' + str(row["POPESTIMATE2019"])
    
    zhvi_county_inc_pop.at[idx, 'text_21'] = 'County:' + row["RegionName"] + '<br>' + 'State:' + row["State"] + \
    '<br>' + '2020-21 increase:' + str(round(row["2021_increase"], 3)) + '<br>' + 'Med_Inc:' + str(row["med_inc"]) + \
        '<br>' + 'Pop_2020:' + str(row["POPESTIMATE2020"])
    
    zhvi_county_inc_pop.at[idx, 'text_2yrs'] = 'County:' + row["RegionName"] + '<br>' + 'State:' + row["State"] + \
    '<br>' + '2019-21 increase:' + str(round(row["2021_2yr_increase"], 3)) + '<br>' + 'Med_Inc:' + str(row["med_inc"]) + \
        '<br>' + 'Pop_2020:' + str(row["POPESTIMATE2020"])



In [25]:
zhvi_county_inc_pop.head()

,SizeRank,RegionName,RegionType,StateName_x,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2019-01-31,2019-02-28,...,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE042020,POPESTIMATE2020,COUNTY_FIPS
0,0,Los Angeles County,County,CA,CA,Los Angeles-Long Beach-Anaheim,6,37,634890.0,632717.0,...,9987189,10033449,10077263,10094865,10092365,10061533,10011602,9968969,9943046,06037
1,1,Cook County,County,IL,IL,Chicago-Naperville-Elgin,17,31,253380.0,253997.0,...,5252513,5254108,5243371,5223386,5199582,5171007,5145326,5121283,5108284,17031
2,2,Harris County,County,TX,TX,Houston-The Woodlands-Sugar Land,48,201,197330.0,198156.0,...,4352419,4454951,4556559,4622836,4655798,4676913,4709243,4734392,4738253,48201
3,3,Maricopa County,County,AZ,AZ,Phoenix-Mesa-Scottsdale,4,13,272509.0,273768.0,...,4018657,4094842,4174423,4258019,4329227,4405306,4492261,4560260,4579081,04013
4,4,San Diego County,County,CA,CA,San Diego-Carlsbad,6,73,590039.0,587680.0,...,3210788,3249307,3280825,3305462,3319019,3332483,3330459,3334413,3332427,06073


In [26]:
zhvi_county_inc_pop.columns
cols_to_keep = ['RegionName','State', 'Metro','FIPS','2021_2yr_increase','text_2yrs','med_inc', 'pov_rate','POPESTIMATE2020']

In [27]:
zhvi_county_inc_pop = zhvi_county_inc_pop[cols_to_keep]
zhvi_county_inc_pop.to_csv("zhvi_county_inc_pop_clean.csv")
